# Introduction of Snorkel
snorkel은 manual labeling이 없이 데이터셋을 프로그래밍틱하게 라벨링하는 방법론임.

스노클은 현재 아래와 같은 세가지의 operation을 제공함.
<img src="https://www.snorkel.org/doks-theme/assets/images/layout/Overview.png">

1. Labeling Data
    - rule 또는 거리기반의 supervision technique을 활용하여 처리함.
2. Transforming Data
    - 데이터 augmenatation
    - NLP에서 효과적인 데이터 augmentation은 어떤 방법일까?
3. Slicing Data
    - 전체 데이터에서 determinant plane에 대한 uncertainty가 높은 부분을 따로 subset하는 방법.

---

## 유튜브 데이터를 활용한 스팸 분류 모델

아래와 같은 기본적인 다섯가지 스텝을 따른다.

1. Writing Labeling Functions (LFs)
- 수작업으로 training data를 라벨링 하는 것이 아닌 프로그래밍적인 방법으로 라벨링을 진행함.

2. Modeling & Combining LFs
- Next, we’ll use Snorkel’s LabelModel to automatically learn the accuracies of our LFs and reweight and combine their outputs into a single, confidence-weighted training label per data point.

3. Writing Transformation Functions (TFs) for Data Augmentation
- Then, we’ll augment this labeled training set by writing a simple TF.

4. Writing Slicing Functions (SFs) for Data Subset Selection
- We’ll also preview writing an SF to identify a critical subset or slice of our training set.

5. Training a final ML model
- Finally, we’ll train an ML model with our training set.


##### Import Modules

In [1]:
import glob
import os
import subprocess

import pandas as pd

# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", 0)

##### Load Data

In [2]:
filenames = sorted(glob.glob("data/Youtube*.csv"))
dfs = []
for i, filename in enumerate(filenames, start=1):
    df = pd.read_csv(filename)
    # Lowercase column names
    df.columns = map(str.lower, df.columns)
    # Rename fields
    df = df.rename(columns={"class": "label", "content": "text"})
    # Remove comment_id, label fields
    df = df.drop("comment_id", axis=1)
    df = df.drop("label", axis=1)
    # Shuffle order
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    dfs.append(df)

df_train = pd.concat(dfs)

In [3]:
df_train.head()

,author,date,text
0,Alessandro leite,2014-11-05T22:21:36,pls http://www10.vakinha.com.br/VaquinhaE.aspx?e=313327 help me get vip gun cross fire al﻿
1,Salim Tayara,2014-11-02T14:33:30,"if your like drones, plz subscribe to Kamal Tayara. He takes videos with his drone that are absolutely beautiful.﻿"
2,Phuc Ly,2014-01-20T15:27:47,go here to check the views :3﻿
3,DropShotSk8r,2014-01-19T04:27:18,"Came here to check the views, goodbye.﻿"
4,css403,2014-11-07T14:25:48,"i am 2,126,492,636 viewer :D﻿"


### 1) Writing Labeling Functions
- Labeling function은 데이터에 대한 휴리스틱과 다양한 규칙을 부여할 수 있음.
    - 즉, 도메인에 대한 사전지식을 바탕으로 labeling을 진행할 수 있음.
- Labeling Function의 핵심 아이디어는 규칙이 완벽하게 정확하지 않아도 된다는 점이다. 심지어 규칙들 간의 높은 상관성도 문제가 없다.(why?)
    - Snorkel이 자동적으로 데이터의 정확도와 상관성을 일관성 있는 방법으로 예측하고, labeling을 달아준다.

In [5]:
import re
from snorkel.labeling import labeling_function
from textblob import TextBlob

In [6]:
# Define the label mappings for convenience
ABSTAIN = -1
NOT_SPAM = 0
SPAM = 1

# 키워드 매칭
@labeling_function()
def lf_keyword_my(x):
    """Many spam comments talk about 'my channel', 'my video', etc."""
    return SPAM if "my" in x.text.lower() else ABSTAIN

# 정규표현식
@labeling_function()
def lf_regex_check_out(x):
    """Spam comments say 'check out my video', 'check it out', etc."""
    return SPAM if re.search(r"check.*out", x.text, flags=re.I) else ABSTAIN

# 임의의 휴리스틱
@labeling_function()
def lf_short_comment(x):
    """Non-spam comments are often short, such as 'cool video!'."""
    return NOT_SPAM if len(x.text.split()) < 5 else ABSTAIN

# Third-party 모델
@labeling_function()
def lf_textblob_polarity(x):
    """
    We use a third-party sentiment classification model, TextBlob.

    We combine this with the heuristic that non-spam comments are often positive.
    """
    return NOT_SPAM if TextBlob(x.text).sentiment.polarity > 0.3 else ABSTAIN

### 2) Combining & Cleaning the Labels
- Unlabeled training data에 우리가 만든 labeling function들을 적용한다.
    - 그러면, label matrix가 나옴.
    - 예를 들면, 이 케이스의 경우 label_function이 4개니까, $(m, 4)$의 label_matrix가 생성됨.
- LabelModel을 활용해서 label들을 reweight and combine해서 하나 label로 예측하고, 통합함.
    - $(m, 4) \longrightarrow (m, 1)$

In [7]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

# Define the set of labeling functions (LFs)
lfs = [lf_keyword_my, lf_regex_check_out, lf_short_comment, lf_textblob_polarity]

# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)

# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

100%|██████████| 1956/1956 [00:01<00:00, 1058.84it/s]


- ABSTAIN은 하나로 labeling하기 애매한 경우에 사용함.

In [10]:
df_train = df_train[df_train.label != ABSTAIN]

### 3) Writing Transformation Functions for Data Augmentation

- 개인적으로 동의어 replace 기반의 Data Augmentation 방법론은 잘 모르겠다.

### 4) Writing a Slicing Function

- 머신러닝 문제에서 데이터셋의 특정 부분이 다른 부분보다 더 중요한 경우가 많음.
- 예를 들면, 유튜브 스팸 필터링 데이터에서 shortened url 패턴이 존재하면 없는 경우보다 스팸일 가능성(likelihood)가 높음.

In [17]:
from snorkel.slicing import slicing_function


@slicing_function()
def short_link(x):
    """Return whether text matches common pattern for shortened ".ly" links."""
    return int(bool(re.search(r"\w+\.ly", x.text)))

- 구체적인 사용방법은 SF tutorial에서 진행함.

### 5) Training a Classifier
- 너가 원하는 모델의 input 데이터로 사용해서 모델링 해봐